In [1]:
import pandas as pd 
import json

# Load json object, and get the Categories array to use as a dataframe
with open('../src/scraped_data/Prayermate.json') as f:
    data = json.load(f)
    categories = data['Categories']

prayerRequests = []
for category in categories:
    for subject in category['subjects']:
        for prayerRequest in subject['cards']:
            prayerRequests.append({
                'category': category['name'],
                'subject': subject['name'],
                'prayerRequest': prayerRequest['text'],
                'archived': prayerRequest['archived'],
                'stackGroup': prayerRequest['stackGroup'],
            })

df = pd.DataFrame(prayerRequests)
df[df['subject'] == 'Jadon'].head()

,category,subject,prayerRequest,archived,stackGroup
373,Monday Bible Study,Jadon,Prayers for Melissa cousin whose wedding got c...,False,ed78b3de-a628-4f17-bfb2-217ed3e3f6df
374,Monday Bible Study,Jadon,Praise for getting focused over weekend on lea...,False,49c4a9cb-5129-4411-b167-488f44e2a49d
375,Monday Bible Study,Jadon,Still very busy season at work,True,b5209420-418d-4d0e-a953-1518665759d5
376,Monday Bible Study,Jadon,Sleep apnea device!,False,5b84f7fd-1fa1-4529-a274-0369affcad37
377,Monday Bible Study,Jadon,Continued prayers for Natalie as she's working...,True,c9c1283a-a030-4aa1-a6a0-a47ca1ae262d


In [2]:
import sys
sys.path.append('..')
from src.semantic_similarity.document_search import DocumentSearchCosSim, DocumentSearchDotProd, HugfaceEmbedder, PrayerRequestCorpus

gteBase = HugfaceEmbedder("thenlper/gte-base")
msmarco = HugfaceEmbedder("msmarco-distilbert-base-dot-prod-v3")

corpus = PrayerRequestCorpus(df, 100)
cachePath = lambda name: f"../cache/prayers/{name}"

embedder_short = DocumentSearchCosSim(gteBase, cachePath("cossim"), corpus)
embedder_long = DocumentSearchDotProd(msmarco, cachePath("dotprod"), corpus)

query = """
Prayers for busy season at work
"""

embedder_short.find_similar(query, top_k=5)
# embedder_long.find_similar(query, top_k=5)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
corpus = PrayerRequestCorpus(df[df["subject"] == "Jadon"], 100)
docStore = DocumentSearchCosSim(gteBase, None, corpus)
docStore.find_similar(query, top_k=2).print_results()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.04it/s]





Top 5 most similar sentences in corpus:
Jadon - Still very busy season at work (Score: 0.8900)
Jadon - Prayer for work: been feeling close to burnout, and giving up emotions to God. Also balance for empa... (Score: 0.8887)
